In [ ]:
#Sentinel-1_cambios por punto de la malla

import ee
ee.Initialize()
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy 
import seaborn as sns
from scipy import stats
from scipy.stats import norm, gamma, f, chi2, lognorm, kstest,fisk
import pandas as pd
import folium
from selenium import webdriver
import time
import math
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import Image
import IPython.display as disp
import imageio
#%matplotlib inline
from sklearn.mixture import GaussianMixture
from openpyxl import load_workbook
import scipy.stats as st
from typing import Optional
import datetime as dt
import re
from pathlib import Path
from openpyxl import load_workbook
from bisect import bisect_left

def append_df_to_excel(filename, df, sheet_name, startrow=None,truncate_sheet=False,**to_excel_kwargs):

    if not os.path.isfile(filename):
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')
    writer.book = load_workbook(filename)
    
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    if truncate_sheet and sheet_name in writer.book.sheetnames:
        idx = writer.book.sheetnames.index(sheet_name)
        writer.book.remove(writer.book.worksheets[idx])
        writer.book.create_sheet(sheet_name, idx)
    
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
    writer.save()

from scipy.stats import (
    norm, beta, expon, gamma, genextreme, logistic, lognorm, triang, uniform, fatiguelife,            
    gengamma, gennorm, dweibull, dgamma, gumbel_r, powernorm, rayleigh, weibull_max, weibull_min, 
    laplace, alpha, genexpon, bradford, betaprime, burr, fisk, f, genpareto, hypsecant, 
    halfnorm, halflogistic, invgauss, invgamma, levy, loglaplace, loggamma, maxwell, 
    mielke, ncx2, ncf, nct, nakagami, pareto, lomax, powerlognorm, powerlaw, rice, 
    semicircular, rice, invweibull, foldnorm, foldcauchy, cosine, exponpow, 
    exponweib, wald, wrapcauchy, truncexpon, truncnorm, t, rdist
    )

distributions = [
    norm, beta, expon, gamma, genextreme, logistic, lognorm, triang, uniform, fatiguelife,            
    gengamma, gennorm, dweibull, dgamma, gumbel_r, powernorm, rayleigh, weibull_max, weibull_min, 
    laplace, alpha, genexpon, bradford, betaprime, burr, fisk, f, genpareto, hypsecant, 
    halfnorm, halflogistic, invgauss, invgamma, levy, loglaplace, loggamma, maxwell, 
    mielke, ncx2, ncf, nct, nakagami, pareto, lomax, powerlognorm, powerlaw, rice, 
    semicircular, rice, invweibull, foldnorm, foldcauchy, cosine, exponpow, 
    exponweib, wald, wrapcauchy, truncexpon, truncnorm, t, rdist
    ]

dist_continu = [d for d in dir(stats) if
                isinstance(getattr(stats, d), stats.rv_continuous)]
dist_discrete = [d for d in dir(stats) if
                 isinstance(getattr(stats, d), stats.rv_discrete)]
print('number of continuous distributions: %d' % len(dist_continu))
print('number of discrete distributions:   %d' % len(dist_discrete))

# función de folium 
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
  
# función de distribución chi2 para serie temporal.
def chi2cdf(chi2, df):
    """Calculates Chi square cumulative distribution function for
       df degrees of freedom using the built-in incomplete gamma
       function gammainc().
    """
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def det(im):
    """Calculates determinant of 2x2 diagonal covariance matrix."""
    return im.expression('b(0)*b(1)')

#### Función ks-test para obtener el valor de P con el dataframe, ver df
def kstest(data, distname, paramtup):
    ksN = 100           # Kolmogorov-Smirnov KS test for goodness of fit: samples    
    ks = scipy.stats.kstest(data, distname, paramtup, ksN)[1] # return p-value
    return ks             # return p-value
# distribution fitter and call to KS test
def fitdist(data1, dist):    
    fitted = dist.fit(data1, floc=0.0)
    ks = kstest(data1, dist.name, fitted)
    res = (dist.name, ks , *fitted)
    return res

# histogramas obtenidos para las distribuciones con valor P > alpha 
def plot_fitted_pdf(df):
    
    N = len(df)
    chrows = math.ceil(N/3) # how many rows of charts if 3 in a row
    fig, ax = plt.subplots(chrows, 3, figsize=(20, 5 * chrows))
    ax = ax.ravel()
    dfRV = pd.DataFrame()
    for i in df.index:
        # D_row = df.iloc[i,:-1]
        D_name = df.iloc[i,0]
        D = df.iloc[i,7]
        KSp = df.iloc[i,1]
        params = df.iloc[i,2:7]    
        params = [p for p in params if ~np.isnan(p)]
# calibrate x-axis by finding the 1% and 99% quantiles in percent point function
        x = np.linspace(
                    D.ppf(0.01, *params), 
                    D.ppf(0.99, *params), 100)
        #fig, ax = plt.subplots(1, 1)
        # plot histogram of actual observations
        ax[i].hist(df, density=True, histtype='stepfilled', alpha=0.2)
        # plot fitted distribution
        rv = D(*params)
        title = f'pdf {D_name}, with p(KS): {KSp:.2f}' 
        ax[i].plot(x, rv.pdf(x), 'r-', lw=2, label=title)
        ax[i].legend(loc="upper right", frameon=False)  

def getCols(tableMetadata):
    return tableMetadata.columns

# This function adds a band representing the image timestamp.
def addTime(image): 
  return image.addBands(image.metadata('system:time_start'))


def coverage_filter(geom,resolution,area,threshold,list_bands):
  
  def MapImageCol (single_image):
    actual_cover_area=(single_image.select(list_bands).clip(geom).reduceRegion(**{'reducer': ee.Reducer.count(),'maxPixels': 1E13}).values().get(0))
    gee_num=ee.Number(actual_cover_area).multiply(resolution).divide(area)

    return ee.Algorithms.If(gee_num.gt(threshold),single_image,ee.Image(0))

  return MapImageCol

def RGB_images(satellite,geom,scene_collection,filepath):             
    acq_times1 = scene_collection.aggregate_array('system:time_start').getInfo()
    dates1=[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times1]  
    
    # Selección de dos imagenes y extracción de las bandas VV
    scene_list = scene_collection.toList(scene_collection.size())
    
    if scene_collection.size().getInfo()==0:
        print('no scenes available for images generation')        
        return [0,'','','','']
    
    elif scene_collection.size().getInfo()==1:
        print('1 scene available')
        
        im1 = ee.Image(scene_list.get(0))
        
        if satellite in ['L8']:
            hsv = im1.select(['B4', 'B3', 'B2']).rgbToHsv()
            sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), im1.select('B8')]).hsvToRgb()
            image_rgb1 = sharpened.visualize(**{'bands': ['red', 'green', 'blue'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
        if satellite in ['L7']:
            hsv = im1.select(['B3', 'B2','B1']).rgbToHsv()
            sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), im1.select('B8')]).hsvToRgb()
            image_rgb1 = sharpened.visualize(**{'bands': ['red', 'green', 'blue'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
    
        if satellite in ['S2']:
            rgb = im1.select(['B4', 'B3', 'B2']).divide(10000)
            image_rgb1 = rgb.visualize(**{'bands': ['B4', 'B3', 'B2'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
        if satellite in ['L4','L5']:
            rgb = im1.select(['B3', 'B2', 'B1'])
            image_rgb1 = rgb.visualize(**{'bands': ['B3', 'B2', 'B1'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
        
        display4 = ee.Image(0).updateMask(0).paint(mined_area,'red',2)
        image_vect_distrubed_zones= display4.visualize(**{'min': 0, 'max': 20, 'palette': ['black', 'black']})    
        
        mosaic1 = ee.ImageCollection([image_rgb1,image_vect_distrubed_zones]).mosaic()
           
        # Create a URL to the styled image for a region around France.
        url = mosaic1.getThumbUrl({'dimensions': [1024,768] , 'region': geom})
                
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        
        try:
          os.makedirs(filepath + '\\' + satellite + '\\')
        except:
          print ("folder already created")
    
        driver.get_screenshot_as_file(filepath + '\\' + satellite + '\\' +'past_image.png')
        driver.quit()
             
        return [1,filepath + '\\' + satellite + '\\' +'past_image.png', dates1[0],'','']
       
    else:
            
        index1 = 0
        index2 = scene_collection.size().getInfo()-1
        
        #past#
        im1 = ee.Image(scene_list.get(index1))
       
        #present#
        im2 = ee.Image(scene_list.get(index2))        
    
        
        if satellite in ['L8']:
            hsv = im1.select(['B4', 'B3', 'B2']).rgbToHsv()
            sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), im1.select('B8')]).hsvToRgb()
            image_rgb1 = sharpened.visualize(**{'bands': ['red', 'green', 'blue'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
            hsv = im2.select(['B4', 'B3', 'B2']).rgbToHsv()
            sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), im2.select('B8')]).hsvToRgb()
            image_rgb2 = sharpened.visualize(**{'bands': ['red', 'green', 'blue'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
        if satellite in ['L7']:
            hsv = im1.select(['B3', 'B2','B1']).rgbToHsv()
            sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), im1.select('B8')]).hsvToRgb()
            image_rgb1 = sharpened.visualize(**{'bands': ['red', 'green', 'blue'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
            hsv = im2.select(['B3', 'B2','B1']).rgbToHsv()
            sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), im2.select('B8')]).hsvToRgb()
            image_rgb2 = sharpened.visualize(**{'bands': ['red', 'green', 'blue'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
    
        if satellite in ['S2']:
            rgb = im1.select(['B4', 'B3', 'B2']).divide(10000)
            image_rgb1 = rgb.visualize(**{'bands': ['B4', 'B3', 'B2'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
            rgb = im2.select(['B4', 'B3', 'B2']).divide(10000)
            image_rgb2 = rgb.visualize(**{'bands': ['B4', 'B3', 'B2'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
        
       
        if satellite in ['L4','L5']:
            rgb = im1.select(['B3', 'B2', 'B1'])
            image_rgb1 = rgb.visualize(**{'bands': ['B3', 'B2', 'B1'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
    
            rgb = im2.select(['B3', 'B2', 'B1'])
            image_rgb2 = rgb.visualize(**{'bands': ['B3', 'B2', 'B1'],'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]})
      
        
        display4 = ee.Image(0).updateMask(0).paint(mined_area,'red',2)
        image_vect_distrubed_zones= display4.visualize(**{'min': 0, 'max': 20, 'palette': ['black', 'black']})    
        
        mosaic1 = ee.ImageCollection([image_rgb1,image_vect_distrubed_zones]).mosaic()
        mosaic2 = ee.ImageCollection([image_rgb2,image_vect_distrubed_zones]).mosaic()
           
        # Create a URL to the styled image for a region around France.
        url = mosaic1.getThumbUrl({'dimensions': [1024,768] , 'region': geom})
                
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)

        try:
          os.makedirs(filepath + '\\' + satellite + '\\')
        except:
          print ("folder already created")
    
        driver.get_screenshot_as_file(filepath + '\\' + satellite + '\\' +'past_image.png')
        driver.quit()
                   
        # Create a URL to the styled image for a region around France.
        url = mosaic2.getThumbUrl({'dimensions': [1024,768] , 'region': geom})
                
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
    
        driver.get_screenshot_as_file(filepath + '\\' + satellite + '\\'  +'present_image.png')
        driver.quit()
    
        
        return [2,filepath + '\\' + satellite + '\\' +'past_image.png', dates1[0], filepath + '\\' + satellite + '\\'  +'present_image.png',dates1[index2]]

def _fecha_de_nombre(zona_id: str) -> dt.date:
    """
    Extrae la fecha  en el nombre de la zona, e.g. 'botadero‑YYYY‑MM‑DD'.
    Devuelve None si el patrón no coincide.
    """
    try:
        y, m, d = zona_id.split('-')[-3:]
        return dt.date(int(y), int(m), int(d))
    except Exception:
        return None

def coverage_filter(geom, min_coverage, area_tot, cob, band):
    def _fn(img):
        return img
    return _fn

def add_inside_flag(puntos_fc, geom_z, zona_id):
    return puntos_fc.map(
        lambda f: f.set({
            'inside': geom_z.contains(f.geometry(), maxError=1),
            'zona_actual': zona_id
        })
    )

def fc_to_dataframe(fc, batch_size=5000) -> pd.DataFrame:
    try:
        size = fc.size().getInfo()
    except Exception:
        size = batch_size
    dfs = []
    for start in range(0, size, batch_size):
        sub_fc   = ee.FeatureCollection(fc.toList(batch_size, start))
        sub_info = sub_fc.getInfo()
        dfs.append(
            pd.json_normalize([f['properties'] for f in sub_info['features']])
        )
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

def multitemporal_bivar_changes_2(
        distrubed_zones: ee.FeatureCollection,
        puntos_fc: ee.FeatureCollection,
        start_date: str = '2017-01-01',
        end_date: str = '2025-12-31',
        filepath: str = r"C:\Users\Natascha\Desktop\Tesis\pantallazos",
        cob: float = 0.95,
        show_map: bool = False
) -> pd.DataFrame:
    os.makedirs(filepath, exist_ok=True)
    out_csv = os.path.join(filepath, 'cambios_punto_a_punto2_15000.csv')

    zonas_info = []
    for f in distrubed_zones.getInfo()['features']:
        zona_id = f['properties']['zona']
        fecha = _fecha_de_nombre(zona_id)
        if fecha:
            zonas_info.append({
                'zona_id': zona_id,
                'fecha': fecha,
                'geom': ee.Feature(f).geometry()
            })
    if not zonas_info:
        print("[WARN] Ninguna zona tiene fecha embebida")
        return pd.DataFrame()

    zonas_info.sort(key=lambda z: z['fecha'])
    fechas_ord = [z['fecha'] for z in zonas_info]
    
    zonas_union = distrubed_zones.geometry().bounds()
    S1_global = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filterBounds(zonas_union)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                 .select(['VV', 'VH'])
                 .sort('system:time_start'))
    total_imgs = S1_global.size().getInfo()
    print(f"Imágenes Sentinel‑1 (DESCENDING, VH) 2017‑25: {total_imgs}")

    def _nearest_zona(fecha_img: dt.date) -> dict:
        pos = bisect_left(fechas_ord, fecha_img)
        cand = []
        if pos:
            cand.append(zonas_info[pos-1])
        if pos < len(fechas_ord):
            cand.append(zonas_info[pos])
        return min(cand, key=lambda z: abs(z['fecha'] - fecha_img))

    def _label_points(c_map, geom_z, zona_id, fecha_img):
        malla_flag = add_inside_flag(puntos_fc, geom_z, zona_id)
        sample = (c_map.unmask(-1).rename('code')
                  .sampleRegions(malla_flag,
                                 properties=['numero', 'inside', 'zona_actual'],
                                 scale=10, tileScale=4))
        df = fc_to_dataframe(sample).rename(columns={'numero': 'punto_id'})
        df['fecha'] = pd.to_datetime(fecha_img)
        df['cambio'] = df['code'].map({-1: 'fuera', 0: 'sin cambios',
                                       1: 'indefinido', 2: 'alzamiento',
                                       3: 'subsidencia'})
        return df[['punto_id', 'zona_actual', 'inside', 'fecha', 'cambio']]

    imgs_list = S1_global.toList(total_imgs)
    fechas_ms = S1_global.aggregate_array('system:time_start').getInfo()
    fechas_dt = [dt.datetime.utcfromtimestamp(ms/1000).date() for ms in fechas_ms]

    header_csv = True
    puntos_global = []

    for k in range(total_imgs - 1):
        im1 = ee.Image(imgs_list.get(k))
        im2 = ee.Image(imgs_list.get(k+1))
        fecha_inicio = fechas_dt[k]
        fecha_fin = fechas_dt[k+1]

        zona_info = _nearest_zona(fecha_fin)
        zona_id = zona_info['zona_id']
        geom_z = zona_info['geom']
        area_tot = geom_z.area(maxError=1).getInfo()

        pair_ic = ee.ImageCollection([im1, im2])
        for band in ['VV']:
            pair_ic = pair_ic.map(coverage_filter(geom_z, 100, area_tot, cob, band))
        if pair_ic.size().getInfo() < 2:
            print(f"[SKIP] {fecha_fin} → pares sin cobertura suficiente")
            continue
        im1 = ee.Image(pair_ic.toList(2).get(0))
        im2 = ee.Image(pair_ic.toList(2).get(1))

        #  cálculo de m2logQ, p_value, c_map 
        def det(im):
            return im.expression('b(0) * b(1)')

        list1 = []
        list2 = []

        for xx in range(0, 31, 1):
            m = 2 + xx * 0.1
            m2logQ = det(im1).log()\
                      .add(det(im2).log())\
                      .subtract(det(im1.add(im2)).log().multiply(2))\
                      .add(4 * np.log(2))\
                      .multiply(-2 * m)

            hist = m2logQ.reduceRegion(
                ee.Reducer.fixedHistogram(0, 20, 200),
                geom_z
            ).get('VV').getInfo()
            aa = np.array(hist)
            x = aa[:, 0]
            y = aa[:, 1] / np.sum(aa[:, 1])
            yyy = (np.abs(y - (chi2.pdf(x, 2) / 10)) /
                   ((y + (chi2.pdf(x, 2) / 10)) / 2)).sum()

            list1.append(m)
            list2.append(yyy)

        opt_array = np.array([list1, list2])
        numb = np.where(opt_array[1, :] == opt_array[1, :].min())[0][0]

        ### óptima
        m = opt_array[0, numb]
        m2logQ = det(im1).log()\
                  .add(det(im2).log())\
                  .subtract(det(im1.add(im2)).log().multiply(2))\
                  .add(4 * np.log(2))\
                  .multiply(-2 * m)

        def chi2cdf(chi2_img, df):
            ''' Chi square CDF using gammainc '''
            return ee.Image(chi2_img.divide(2)).gammainc(ee.Number(df).divide(2))

        p_value = ee.Image.constant(1).subtract(chi2cdf(m2logQ, 2))

        hist = p_value.reduceRegion(ee.Reducer.fixedHistogram(0, 1, 100),geom_z).get('constant').getInfo()
        aa = np.array(hist)
        x = aa[:, 0]
        y = aa[:, 1] / np.sum(aa[:, 1])

        plt.plot(x, y, '.b', label='p-value')
        plt.ylim(0, 0.05)
        plt.grid()
        plt.legend()
        plt.show()

        c_map = p_value.multiply(0).where(p_value.lt(0.05), 1)
        diff = im2.subtract(im1)
        d_map = c_map.multiply(0)                    # 0 sin cambios
        d_map = d_map.where(det(diff).gt(0), 2)      # 2 alzamientos
        d_map = d_map.where(diff.select(0).gt(0), 3) # 3 subsidencias
        d_map = d_map.where(det(diff).lt(0), 1)      # 1 indefinido
        c_map = c_map.multiply(d_map)

        if show_map:
            import geemap, webbrowser, pathlib

            Map = geemap.Map(height=600)
            Map.centerObject(zona_info['geom'], 13)

            changed_clip = c_map.select('constant').clip(zona_info['geom'])

            vis = {
                'min': 0, 'max': 3,
                'palette': ['black', 'red', 'cyan', 'yellow']
            }
            Map.addLayer(
                changed_clip,
                vis,
                f'Cambios {zona_id} '
                f'{fecha_inicio.strftime("%Y-%m-%d")} → '
                f'{fecha_fin.strftime("%Y-%m-%d")}',
                shown=True
            )

            Map.addLayer(zona_info['geom'], {'color': 'white'}, 'Límite zona_actual', True)

            nombre_html = os.path.join(
                filepath,
                f'cambios_{zona_id}_'
                f'{fecha_inicio.strftime("%Y%m%d")}_'
                f'{fecha_fin.strftime("%Y%m%d")}.html'
            )
            Map.to_html(nombre_html)
            print(f"[INFO] Mapa guardado en {nombre_html}")
            try:
                os.startfile(nombre_html)
            except AttributeError:
                webbrowser.open(pathlib.Path(nombre_html).resolve().as_uri())
                
        df_pts = _label_points(c_map, geom_z, zona_id, fecha_fin)
        if not df_pts.empty:
            df_pts.to_csv(out_csv, mode='a', header=header_csv, index=False)
            header_csv = False
            puntos_global.append(df_pts)


    if puntos_global:
        return pd.concat(puntos_global, ignore_index=True)
    print("[INFO] No se generaron puntos.")
    return pd.DataFrame()


dire=r"C:\Users\carpeta"


malla_de_puntos =  ee.FeatureCollection("projects/ee-projects/assets/15000_numerados")
Feature1 = ee.FeatureCollection("projects/ee-projects/assets/2023")
a=Feature1.first()

col_names=pd.DataFrame.from_dict(a.toDictionary(a.propertyNames()).getInfo(), orient='index').reset_index()
col_names=list(col_names['index'])

Feature1.aggregate_count_distinct('Name').getInfo()


for Muestra in list(pd.Series(Feature1.aggregate_array('Name').getInfo()).unique()):
    
    mined_area = Feature1.filter(ee.Filter.stringContains('Name',"area_mina"))
    mined_area.aggregate_array('COD_SSUBC').getInfo()
    vector = mined_area.geometry().bounds()
    filepath=dire + '\\' + Muestra + '\\'
    try:
      os.makedirs(dire + '\\' + Muestra )
    except:
      print ("folder already created")
    final_date='2025-06-13'
    ALPHA = 0.05        
    error=5
    cob = 0.95
    buffer=250
    significancia=0.95
    distrubed_zones = ee.FeatureCollection("projects/ee-projects/assets/botadero2017-2025")
    

def main() -> None:

    try:
        ee.Initialize(project='ee-projects')
    except ee.EEException:
        ee.Authenticate()
        ee.Initialize(project='ee-projects')

    START = '2017-01-01'
    END   = '2025-12-31'

    OUT_DIR = Path(r"C:\Users\Natascha\Desktop\Tesis\pantallazos")
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    ZONAS_FC = ee.FeatureCollection(
        "projects/ee-projects/assets/botadero2017-2025"
    )
    PUNTOS_FC = ee.FeatureCollection(
        "projects/ee-projects/assets/15000_numerados"
    )

    df_result = multitemporal_bivar_changes_2(
        distrubed_zones = ZONAS_FC,
        puntos_fc       = PUNTOS_FC,
        start_date      = START,
        end_date        = END,
        filepath        = str(OUT_DIR),
        cob             = 0.95,
        show_map        = True   
    )

    if not df_result.empty:
        csv_path = OUT_DIR / "resumen_puntos_global_0.05.csv"
        df_result.to_csv(csv_path, index=False, encoding='utf-8')
        print(f"\n✓ DataFrame global guardado en: {csv_path}")

if __name__ == "__main__":
    main()



In [ ]:
# Extraer la informacion de cada banda de cada satelite (L8,L9, S2) en cada punto_id 

import ee
import datetime as dt
import math
import pandas as pd
import re
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple

ee.Initialize()

ZONAS_ASSET = "projects/ee-projects/assets/botadero2017-2025"
MALLA_ASSET = "projects/ee-projects/assets/nueva_malla_num_xy"

OUT_DIR = Path(r"C:\Users\carpeta")
OUT_DIR.mkdir(exist_ok=True)

FECHA_INI = "2016-01-01"
FECHA_FIN = "2025-12-31"
NUBE_MAX = 40

PIXEL_COUNT = 15_000
MIN_COVERAGE_FRAC = 0.90
MIN_VALID_PIXELS = int(PIXEL_COUNT * MIN_COVERAGE_FRAC)  

BATCH_SIZE_DESCARGA = 10590 
TILE_SCALE = 4            
SCALE_VALID = 30         

DO_L7 = True
DO_L8 = True
DO_L9 = True
DO_S2 = True

BANDAS_L7_SR = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B7"]
BANDAS_L7_TH = ["ST_B6"]
BANDAS_L7 = BANDAS_L7_SR + BANDAS_L7_TH

BANDAS_L8_SR = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]
BANDAS_L8_TH = ["ST_B10"]
BANDAS_L8 = BANDAS_L8_SR + BANDAS_L8_TH

BANDAS_S2 = [
    "B1",
    "B2",
    "B3",
    "B4",
    "B5",
    "B6",
    "B7",
    "B8",
    "B8A",
    "B9",
    "B11",
    "B12",
]

def extraer_fecha(nombre: str) -> Optional[dt.date]:
    """Extrae fecha YYYY‑MM‑DD o DD‑MM‑YYYY de un string."""
    m = re.search(r"(\d{4})[-_](\d{2})[-_](\d{2})", nombre)
    if m:
        y, mo, d = map(int, m.groups())
        return dt.date(y, mo, d)
    m = re.search(r"(\d{2})[-_](\d{2})[-_](\d{4})", nombre)
    if m:
        d, mo, y = map(int, m.groups())
        return dt.date(y, mo, d)
    return None

def preparar_zonas(zonas_fc: ee.FeatureCollection) -> List[Dict]:
    """Descarga una vez las zonas y retorna lista con nombre, fecha y geom."""
    zonas_list = zonas_fc.getInfo()["features"]
    out: List[Dict] = []
    for z in zonas_list:
        nombre = z["properties"].get("zona") or z["properties"].get("name", "")
        fz = extraer_fecha(nombre)
        if fz:
            out.append({
                "nombre": nombre,
                "fecha": fz,
                "geom": ee.Geometry(z["geometry"]),
            })
    out.sort(key=lambda d: d["fecha"])
    if not out:
        raise ValueError("No se pudieron extraer fechas válidas de las zonas.")
    return out

def escalar_l7(img: ee.Image) -> ee.Image:
    sr = img.select(BANDAS_L7_SR).multiply(0.0000275).add(-0.2).clamp(0, 1)
    tc = (img.select("ST_B6")
            .multiply(0.00341802)
            .add(149)
            .subtract(273.15)
            .rename("Temp_C"))
    scaled = sr.addBands(tc).copyProperties(img, img.propertyNames())
    return ee.Image(scaled)           

def escalar_l8l9(img: ee.Image) -> ee.Image:
    sr = img.select(BANDAS_L8_SR).multiply(0.0000275).add(-0.2).clamp(0, 1)
    tc = (img.select("ST_B10")
            .multiply(0.00341802)
            .add(149)
            .subtract(273.15)
            .rename("Temp_C"))
    scaled = sr.addBands(tc).copyProperties(img, img.propertyNames())
    return ee.Image(scaled)

def escalar_s2(img: ee.Image) -> ee.Image:
    scaled = img.select(BANDAS_S2).divide(10000) \
                .copyProperties(img, img.propertyNames())
    return ee.Image(scaled)

def add_valid_and_date(img: ee.Image, band_ref: str, geom: ee.Geometry) -> ee.Image:
    mask = img.select([band_ref]).mask().gt(0)
    nval = mask.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=SCALE_VALID,  
        maxPixels=1e9,
    ).get(band_ref)
    return img.set({"n_valid": nval, "date_only": img.date().format("YYYY-MM-dd")})


def preparar_coleccion(
    collection_id: str,
    bandas: List[str],
    nube_prop: str,
    geom_bbox: ee.Geometry,
) -> ee.ImageCollection:
    band_ref = bandas[0]
    return (
        ee.ImageCollection(collection_id)
        .filterDate(FECHA_INI, FECHA_FIN)
        .filterBounds(geom_bbox)
        .filter(ee.Filter.lt(nube_prop, NUBE_MAX))
        .select(bandas)
        .map(lambda im: add_valid_and_date(im, band_ref, geom_bbox))
        .filter(ee.Filter.gte("n_valid", MIN_VALID_PIXELS))
        .sort("n_valid", False)
        .distinct("date_only")
    )

def descargar_fc_csv(fc: ee.FeatureCollection, columnas: List[str]) -> pd.DataFrame:
    url = fc.getDownloadURL(
        filetype="csv",         
        selectors=columnas,    
        filename="tmp"          
    )
    return pd.read_csv(url)

def muestrear_imagenes(
    col: ee.ImageCollection,
    escalar_fn,
    nube_prop: str,
    bandas_finales: List[str],
    mallas_zona: Dict[str, ee.FeatureCollection],
    zonas_info: List[Dict],
    sensor_tag: str,
) -> pd.DataFrame:
    ids = col.aggregate_array("system:index").getInfo()
    print(f"\n[{sensor_tag}] Imágenes filtradas: {len(ids)}")

    filas_acumuladas: List[pd.DataFrame] = []
    t_global = time.time()

    for i, idx in enumerate(ids, 1):
        print(f"    → ({i}/{len(ids)}) comenzando {sensor_tag}  {idx}")
        t0 = time.time()
        im = ee.Image(col.filter(ee.Filter.eq("system:index", idx)).first())
        fecha_ms = im.date().millis().getInfo()
        fecha_img = dt.datetime.utcfromtimestamp(fecha_ms / 1_000).date()
        zsel = min(zonas_info, key=lambda z: abs((z["fecha"] - fecha_img).days))
        malla_etq = mallas_zona[zsel["nombre"]]
        img_sc = ee.Image(escalar_fn(im)) 
        samp = (
            img_sc.sampleRegions(
                collection=malla_etq,
                properties=["numero", "zona_actual", "inside"],
                scale=30,
                tileScale=TILE_SCALE,
            )
            .map(
                lambda f: f.set(
                    {
                        "fecha_img": im.date().format("YYYY-MM-dd"),
                        "cloud": im.get(nube_prop),
                        "sensor": sensor_tag,
                    }
                )
            )
        )

        columnas = (
            ["numero", "zona_actual", "inside", "fecha_img", "cloud", "sensor"]
            + bandas_finales
            + ["Temp_C"]
        )
        df = descargar_fc_csv(samp, columnas).rename(columns={"numero": "punto_id"})

        filas_acumuladas.append(df)

        print(
            f"       filas: {len(df):,} | fecha {fecha_img} | zona {zsel['nombre']} | {time.time() - t0:0.1f}s"
        )

    df_final = pd.concat(filas_acumuladas, ignore_index=True) if filas_acumuladas else pd.DataFrame()
    print(f"[{sensor_tag}] TOTAL filas: {len(df_final):,} | Tiempo total: {time.time() - t_global:0.1f}s")
    return df_final


def main():
    zonas_fc = ee.FeatureCollection(ZONAS_ASSET)
    malla_fc = ee.FeatureCollection(MALLA_ASSET)
    
    zonas_info = preparar_zonas(zonas_fc)
    print(
        f"Zonas válidas: {len(zonas_info)} (ej: {zonas_info[0]['nombre']} → {zonas_info[0]['fecha']})"
    )

    bbox = zonas_fc.geometry()

    mallas_zona = {
        z["nombre"]: malla_fc.map(
            lambda pt, g=z["geom"], n=z["nombre"]: pt.set(
                {"inside": g.contains(pt.geometry()), "zona_actual": n}
            )
        )
        for z in zonas_info
    }

    resultados: List[Tuple[str, pd.DataFrame]] = []


    #if DO_L7:
     #   col = preparar_coleccion("LANDSAT/LE07/C02/T1_L2", BANDAS_L7, "CLOUD_COVER", bbox)
    #    df = muestrear_imagenes(
    #        col, escalar_l7, "CLOUD_COVER", BANDAS_L7_SR, mallas_zona, zonas_info, "L7"
      #  )
    #    resultados.append(("valores_L7_puntos2_15000.csv", df))

    if DO_L8:
        col = preparar_coleccion("LANDSAT/LC08/C02/T1_L2", BANDAS_L8, "CLOUD_COVER", bbox)
        df = muestrear_imagenes(
            col, escalar_l8l9, "CLOUD_COVER", BANDAS_L8_SR, mallas_zona, zonas_info, "L8"
        )
        resultados.append(("valores_L8_puntos_15000.csv", df))

    if DO_L9:
        col = preparar_coleccion("LANDSAT/LC09/C02/T1_L2", BANDAS_L8, "CLOUD_COVER", bbox)
        df = muestrear_imagenes(
            col, escalar_l8l9, "CLOUD_COVER", BANDAS_L8_SR, mallas_zona, zonas_info, "L9"
        )
        resultados.append(("valores_L9_puntos_15000.csv", df))

    if DO_S2:
        col = preparar_coleccion(
            "COPERNICUS/S2_SR_HARMONIZED",
            BANDAS_S2,
            "CLOUDY_PIXEL_PERCENTAGE",
            bbox,
        )
        df = muestrear_imagenes(
            col, escalar_s2, "CLOUDY_PIXEL_PERCENTAGE", BANDAS_S2, mallas_zona, zonas_info, "S2"
        )
        resultados.append(("valores_S2_puntos_15000.csv", df))

    for nombre, df in resultados:
        out_path = OUT_DIR / nombre
        df.to_csv(out_path, sep=";", decimal=".", index=False)
        print(f" Guardado: {out_path} ({len(df)} filas)")


if __name__ == "__main__":
    main()


In [ ]:
# DEM

import ee, pandas as pd
from pathlib import Path
ee.Initialize()

dem = ee.Image("NASA/NASADEM_HGT/001")
malla = ee.FeatureCollection("projects/ee-nataschavonsengerloeper123/assets/nueva_malla_num_xy")

sample = dem.sampleRegions(
    collection = malla,
    properties = ['numero','zona_actual'], 
    scale      = 30,   # NASADEM es a 30 m
    tileScale  = 4
)

count      = sample.size().getInfo()
batch_size = 5000
props = []
for offset in range(0, count, batch_size):
    batch = sample.toList(batch_size, offset).getInfo()
    props.extend(feat['properties'] for feat in batch)

df = pd.json_normalize(props)
out_csv = Path(r"C:\Users\malla_elevacion_NASADEM.csv")
df.to_csv(out_csv, index=False)
print(f"Elevaciones guardadas en {out_csv} ({len(df):,} puntos)")

#  DEM GLO-30 
dem = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic()  
malla = ee.FeatureCollection("projects/ee-projects/assets/nueva_malla_num_xy")

sample = dem.sampleRegions(
    collection = malla,
    properties = ['numero','zona_actual'],  
    scale      = 30,
    tileScale  = 4
)

count      = sample.size().getInfo()
batch_size = 5000

props = []
for offset in range(0, count, batch_size):
    batch = sample.toList(batch_size, offset).getInfo()
    props.extend(f['properties'] for f in batch)

# 4) DataFrame y CSV
df = pd.json_normalize(props)
out_csv = Path(r"C:\Users\\malla_elev_GLO30.csv")
df.to_csv(
    out_csv,
    sep=';',           
    decimal='.',       
    float_format='%.3f',  
    index=False
)
print(f"CSV exportado en: {out_csv}")


In [ ]:
# Datos Clima

from __future__ import annotations
import os
import ee
import pandas as pd
from pathlib import Path
from typing import Optional

def weather_eras(dire: str,
                 Feature1: ee.FeatureCollection,
                 Muestra: str,
                 initial_date: str,
                 final_date: str) -> pd.DataFrame:
    print("→ Starting weather_eras")
    filepath = os.path.join(dire, Muestra, 'weather_eras')
    os.makedirs(filepath, exist_ok=True)
    print(f"   * Output folder: {filepath}")
    filt = ee.Filter.inList('zona', [Muestra])
    geom = Feature1.filter(filt).first().geometry()
    col = (ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
           .filterDate(initial_date, final_date)
           .filterBounds(geom))
    ids = col.aggregate_array('system:index').getInfo()
    print(f"   * {len(ids)} daily scenes between {initial_date} and {final_date}")

    # Bandas diarias
    daily_bands = [
        'dewpoint_temperature_2m','temperature_2m','skin_temperature',
        'soil_temperature_level_1','soil_temperature_level_2',
        'soil_temperature_level_3','soil_temperature_level_4',
        'lake_bottom_temperature','lake_ice_depth','lake_ice_temperature',
        'lake_mix_layer_depth','lake_mix_layer_temperature','lake_shape_factor',
        'lake_total_layer_temperature','snow_albedo','snow_cover','snow_density',
        'snow_depth','snow_depth_water_equivalent','snowfall_sum','snowmelt_sum',
        'temperature_of_snow_layer','skin_reservoir_content',
        'volumetric_soil_water_layer_1','volumetric_soil_water_layer_2',
        'volumetric_soil_water_layer_3','volumetric_soil_water_layer_4',
        'forecast_albedo','surface_latent_heat_flux',
        'surface_net_solar_radiation','surface_net_thermal_radiation',
        'surface_sensible_heat_flux','surface_solar_radiation_downwards',
        'surface_thermal_radiation_downwards','evaporation_from_bare_soil',
        'evaporation_from_open_water_surfaces_excluding_oceans',
        'evaporation_from_the_top_of_canopy',
        'evaporation_from_vegetation_transpiration',
        'potential_evaporation_sum','runoff_sum','snow_evaporation_sum',
        'sub_surface_runoff_sum','surface_runoff_sum','total_evaporation_sum',
        'u_component_of_wind_10m','v_component_of_wind_10m','surface_pressure',
        'total_precipitation_sum','leaf_area_index_high_vegetation',
        'leaf_area_index_low_vegetation'
    ]
    
    records: list[dict] = []

    for idx in ids:
        print(f"Processing {idx}")
        img = ee.Image(f'ECMWF/ERA5_LAND/DAILY_AGGR/{idx}').clip(geom)
        red = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=100,
            bestEffort=True
        ).getInfo()
        red.update({
            'year':  int(idx[:4]),
            'month': int(idx[4:6]),
            'day':   int(idx[6:8]),
            'stat':  'mean'
        })
        records.append(red)

    df = pd.DataFrame(records, columns=daily_bands + ['year','month','day','stat'])
    
    out_path = Path(filepath) / 'weather_daily_tot.xlsx'
    df.to_excel(out_path, index=False)
    print(f"   * Saved to {out_path}")
    return df

def get_largest_feature_name(asset_path: str, name_prop: str = 'zona') -> Optional[str]:
    print("→ Selecting feature with largest area...")
    fc = ee.FeatureCollection(asset_path)
    with_areas = fc.map(lambda f: f.set('area_m2', f.geometry().area(1)))
    largest = ee.Feature(with_areas.sort('area_m2', False).first())
    name = largest.get(name_prop).getInfo()
    print(f"   * Selected: {name}")
    return name

def main():
    dire = r"C:\Users\carpeta"
    asset = 'projects/ee-projects/assets/botadero2017-2025'
    initial = '2016-01-01'
    final   = '2025-12-31'
    
    os.makedirs(dire, exist_ok=True)
    zona = get_largest_feature_name(asset, name_prop='zona')
    print(f" Running weather_eras for zone: {zona}")
    df_wea = weather_eras(dire, ee.FeatureCollection(asset), zona, initial, final)

    print("Finished. Sample output:")
    print(df_wea.head())

if __name__ == "__main__":
    main()
